In [ ]:
import algorithm_states, common

In [3]:
# Lift an element of SO(q0)(k) to SL2 x SL2(k)
def lemma_2_6(m, k = QQ.algebraic_closure()):
    state = algorithm_states.SL2SL2State(m0, output = False)
    
    if state.m[0, 0] == 0 and state.m[0, 1] == 0:
        state.add_row(2, 0)
        
    if state.m[1, 0] == 0:
        state.add_row(0, 1)
        
    state.add_row(1, 0, scale = (1 - state.m[0, 0]) / state.m[1, 0])
    
    state.add_row(0, 1, scale = -state.m[1, 0])
    
    state.add_row(0, 2, scale = -state.m[2, 0])
    
    state.add_row(3, 2, scale = -state.m[2, 3])
    
    state.add_row(3, 1, scale = -state.m[1, 3])
    
    mu = k(sqrt(state.m[1, 1]))
    state.step((diagonal_matrix([1/mu, mu]), diagonal_matrix([mu, 1/mu])))
    
    (lift1, lift2) = state.t
    
    return lift1.inverse(), lift2.inverse()

In [ ]:
# Lift an element of SLn(ZZ / l ZZ) to SLn(ZZ)
def lemma_3_1(m):
    state = algorithm_states.SLnState(m, output = False)
    
    for j in range(0, state.dim - 1):
        while len([i for i in range(state.dim) if state.m[j, i] != 0]) > 1:
            columns = sorted([(ZZ(state.m[j, i]), i) for i in range(state.dim)], reverse = True)
            source = columns[1][1]
            target = columns[0][1]
            factor = columns[0][0] // columns[1][0]
            state.add_column(source, target, scale = -factor, description = f"Subtract {factor} times column {source} from {target}")

        if state.m[j, state.dim - 1] == 0:
            i = [i for i in range(state.dim) if state.m[j, i] != 0][0]
            state.switch_columns(i, state.dim - 1, description = f"Switch columns {i} and {state.dim - 1}")

        state.add_column(state.dim - 1, j, scale = ZZ(1 / state.m[j, state.dim - 1]), description = f"Make entry ({j}, {j}) equal to 1")

        state.add_column(j, state.dim - 1, scale = -state.m[j, state.dim - 1], description = f"Make entry ({j}, {state.dim - 1}) equal to 0")

        for i in range(j + 1, state.dim):
            state.add_row(j, i, scale = -state.m[i, j], description = f"Make entry ({i}, {j}) equal to zero")
            
    left_lift, right_lift = state.t
    return left_lift.inverse() * right_lift.inverse()

In [2]:
# Diagonalize a model over Zp
def lemma_4_3(m, p):
    state = algorithm_states.ModelState(m, output = False)
    a = 1
    
    for i in range(0, state.dim - 1):
        (o, j) = min((common.ord_vector(p, state.m, j), j) for j in range(i, state.dim))
        if o < common.ord_vector(p, state.m, i):
            state.switch_columns(j, i, description = f"Make sure that the order of column {i} is minimal")

        (o, j) = min((common.ord(p, state.m[j, i]), j) for j in range(i, state.dim))
        for k in range(2):
            if o < common.ord(p, state.m[i, i]):
                state.add_row(j, i, description = f"Make sure that the order of entry ({i}, {i}) is minimal")

        a = lcm(a, (state.m[i, i] * p^(-common.ord(p, state.m[i, i]))).numerator())

        for j in range(i + 1, state.dim):
            state.add_row(i, j, -state.m[i, j] / state.m[i, i], f"Make entry ({i}, {j}) equal to 0")
        
    return state.m, state.t, a

In [ ]:
# Lift a model improvement over Zp to a model improvement over Z
def lemma_4_5(m, t, p):
    state = algorithm_states.ModelImprovementState(m, t, output = False)
    s = []
    
    for i in range(0, state.dim):
        (o, j) = min((common.ord_vector(p, state.improvement, j), j) for j in range(i, state.dim))
        if o < common.ord_vector(p, state.improvement, i):
            state.switch_columns(j, i, description = f"Make the order of column {i} minimal")

        s.append(common.ord_vector(p, state.improvement, i))

        (o, j) = min((common.ord(p, state.improvement[j, i]), j) for j in range(i, state.dim))
        if o < common.ord(p, state.improvement[i, i]):
            state.switch_rows(j, i, description = f"Make the order of entry ({i}, {i}) minimal")

        state.step((identity_matrix(state.dim), elementary_matrix(QQ, state.dim, row1=i, scale=p^s[i] / state.improvement[i, i])), description = f"Convert entry ({i}, {i}) to the form p^s")

        for j in range(i + 1, state.dim):
            state.add_column(i, j, -state.improvement[i, j] / state.improvement[i, i], f"Make entry ({i}, {j}) equal to 0")
            
    s_matrix = diagonal_matrix(QQ, [p^n for n in s])
    r = (state.improvement * s_matrix.inverse()).apply_map(lambda x: ZZ(Integers(p^(-2 * s[0]))(x)))
    return state.t[0].inverse() * r, s_matrix

In [ ]:
# Construct a model improvement with respect to p != 2, given that (-1)^(k / 2) times the diagonal elements divisible by p is a square mod p
def theorem_4_1(m, p):
    state = algorithm_states.ModelState(m, output = False)
    k = common.ord(p, state.m.determinant())
    F = GF(p)

    (_, diagonal_transform, _) = lemma_4_3(state.m, p)
    state.step(diagonal_transform, "Rediagonalize")

    state.step(diagonal_matrix(1 / p^((common.ord(p, state.m[i, i]) // 2)) for i in range(state.dim)), "Remove excess square powers of p")

    while min(i for i in range(state.dim) if state.m[i, i] % p != 0) < max(i for i in range(state.dim) if state.m[i, i] % p == 0):
        i = min(i for i in range(state.dim) if state.m[i, i] % p != 0)
        j = max(i for i in range(state.dim) if state.m[i, i] % p == 0)
        state.switch_rows(j, i, "Move entries divisible by p to the front")

    k = common.ord(p, state.m.determinant())
    
    while k >= 4:
        # Find a solution
        solution = None
        if F(-state.m[2, 2] / state.m[1, 1]).is_square():
            solution = (0, 1, ZZ(F(-state.m[2, 2] / state.m[1, 1]).sqrt()))
        else:
            for x2 in range((p + 1) / 2):
                if F(-(state.m[0, 0] + x2^2 * state.m[1, 1]) / state.m[2, 2]).is_square():
                    solution = (1, x2, ZZ(F(-(state.m[0, 0] + x2^2 * state.m[1, 1]) / state.m[2, 2]).sqrt()))
                    break

        if solution[0] == 0:
            state.switch_rows(1, 0, description = "Make the first coefficient of the solution nonzero")
            solution = (solution[1], solution[0], solution[2])

        state.add_row(1, 0, ZZ(solution[1]), "Apply partial improvement")
        state.add_row(2, 0, ZZ(solution[2]), "Apply partial improvement")
        state.step(elementary_matrix(QQ, state.dim, row1 = 0, scale = 1 / p))

        (_, diagonal_transform, _) = lemma_4_3(state.m, p)
        state.step(diagonal_transform, "Rediagonalize")

        state.step(diagonal_matrix(1 / p^((common.ord(p, state.m[i, i]) // 2)) for i in range(state.dim)), "Remove excess square powers of p")

        while min(i for i in range(state.dim) if state.m[i, i] % p != 0) < max(i for i in range(state.dim) if state.m[i, i] % p == 0):
            i = min(i for i in range(state.dim) if state.m[i, i] % p != 0)
            j = max(i for i in range(state.dim) if state.m[i, i] % p == 0)
            state.switch_rows(j, i, "Move entries divisible by p to the front")

        k = common.ord(p, state.m.determinant())
    
    if k == 2:
        l = ZZ(F(-state.m[0, 0] / state.m[1, 1]).sqrt())
        state.add_row(1, 0, l, "Apply partial improvement")
        state.step(elementary_matrix(QQ, state.dim, row1 = 0, scale = 1 / p))

        (_, diagonal_transform, _) = lemma_4_3(state.m, p)
        state.step(diagonal_transform, "Rediagonalize")
        k = common.ord(p, state.m.determinant())
    return state.m, state.t

In [ ]:
# Construct a model improvement with respect to 2, given that the matrix is diagonal and half of the diagonal elements are 1 mod 4 and the other half are 3 mod 4
def lemma_4_7(m):
    state = algorithm_states.ModelState(m, output = False)
    
    while len([i for i in range(0, state.dim) if (state.m[i, i] + 2 * i) % 4 != 1]) > 0:
        i = [i for i in range(0, state.dim) if (state.m[i, i] + 2 * i) % 4 != 1][0]
        j = [j for j in range(i, state.dim) if (state.m[j, j] + 2 * i) % 4 == 1][0]
        state.switch_rows(j, i, description = f"Make sure that entry ({i}, {i}) has the correct parity")
        
    for i in range(0, state.dim // 2):
        state.add_row(2 * i, 2 * i + 1, description = f"Add column {2 * i} to {2 * i + 1}")
        state.step(elementary_matrix(QQ, state.dim, row1 = 2 * i + 1, scale = 1 / 2), description = f"Divide a factor 2 from column {2 * i + 1}")
    
    return state.m, state.t

In [ ]:
# Construct model improvements over different primes and combine them
def lemma_4_8(m):
    dim = m.nrows()
    
    improvements = []
    l = 0
    
    for (p, n) in m.determinant().factor():
        if n % 2 != 0:
            continue
        coefficients = [m[i, i] // p for i in range(dim) if m[i, i] % p == 0]
        if not GF(p)((-1)^(n // 2) * prod(coefficients)).is_square():
            continue
        (_, improvement) = theorem_4_1(m, p)
        r, s = lemma_4_5(m, improvement, p)
        improvements.append((p, r, s))
        l = max(l, -min(common.ord(p, s[i, i]) for i in range(dim)))
    
    if len([i for i in range(dim) if m[i, i] % 4 == 1]) == len([i for i in range(dim) if m[i, i] % 4 == 3]):
        (_, improvement) = algorithms.lemma_4_7(m)
        r, s = algorithms.lemma_4_5(m, improvement, 2)
        improvements.append((2, r, s))
        l = max(l, -min(common.ord(2, s[i, i]) for i in range(dim)))
    
    total_modulus = prod(x[0]^(2 * l) for x in improvements)
    
    r = algorithms.lemma_3_1(matrix(
        [Integers(total_modulus)(CRT_list([x[1][j, i] for x in improvements], [x[0]^(2 * l) for x in improvements])) for i in range(dim)] for j in range(dim)
    ))
    
    t = r * prod(x[2] for x in improvements)
    
    return t.transpose() * m * t, t